In [1]:
%alias_magic t timeit
%matplotlib inline
import json,io
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os,sys
from twitter_analytics_api import *
from datetime import datetime as dt
# Import unquote to prevent url encoding errors in next_results
from urllib import unquote
sys.path.append(os.getcwd()) #sys.argv[0]

# pandas settings 
pd.set_option('expand_frame_repr', False)
pd.options.display.max_columns = 15
pd.options.display.max_rows = 25
pd.set_option('max_colwidth', 30)
pd.set_option('colheader_justify', 'center')
dfm = pd.DataFrame

timer = lambda: '%s_%s_%s'%dt.utctimetuple(dt.now())[3:6]
file_ext = lambda x: dict(to_json='.json',to_csv='.csv',to_excel='.xlsx').get(x,'')

Created `%t` as an alias for `%timeit`.
Created `%%t` as an alias for `%%timeit`.


In [37]:
world_trends = tweet.trends(1,output_format='to_json')
tweet.save()

world_trends = tweet.trends(1,output_format='to_json')
tweet.save(world_trends)
# print json.dumps(world_trends, )

# US_WOE_ID = 23424977
# us_trends = tweet.trends(US_WOE_ID)
# print json.dumps(us_trends, indent=1)

In [8]:
# Sample usage
q = "surgical strike"
# results = tweet.search(q, max_results=1000,output_format='to_data_frame')
# tweet.save()
# 
results = tweet.search(q, max_results=1000,output_format='to_json')
tweet.save()

In [3]:
pproc = lambda x: ',,,'.join(x).lower()
src = r'''



D:\intel\intel\webCrawler\twitter_ana\uint_7\Mining-the-Social-Web-2nd-Edition-master\ipynb\default\20_54_26_search_surgical strike.json




'''
src = src.split('\n')
src = [src for src in src if src.strip() and not src.strip().startswith('#')][0]
statuses_df = pd.read_json(src)
statuses = json.loads(open(src).read())

In [7]:
# %%timeit
tweets = ((status['text'], status['entities']['user_mentions'], status['entities']['hashtags'],
           status['retweet_count'], status.get('retweeted_status'))
                    for status in statuses)
tweets = [
            ( 
                pproc(texts.split()),
                pproc([i['screen_name'] for i in user_mentions]),
                pproc([i['text'] for i in hashtags]),
                retweet_count,
                pproc([retweeted_status['user']['screen_name']]) if retweeted_status else np.nan
            )
                for texts,user_mentions, hashtags, retweet_count, retweeted_status in tweets
        ]
df = dfm(tweets,columns=['texts','screen_names','hashtags','retweet_counts','retweet_screen_names']).drop_duplicates()
# print df.to_csv("output.csv",encoding='utf-8')

None


In [16]:
# %%timeit
# apply works on row / column basis      axis = 0 -> col (vertical), axis = 1 -> row (horizontal)  
# applymap works on element-wise                DataFrame
# map works on element-wise                     Series
tweets_words = {}
def word_counter(col):
    results = []
    col.str.split(',,,').apply(results.extend)
    tweets_words[col.name] = pd.Series(results)
df[['texts','screen_names','hashtags']].apply(word_counter)
print tweets_words['texts'].value_counts()

In [39]:
# A function for computing lexical diversity
def lexical_diversity(tokens):
    return 1.0*len(set(tokens))/len(tokens) 

# A function for computing the average number of words per tweet
average_words = lambda x: sum(x.str.len()/len(x))

print lexical_diversity(tweets_words['texts'])
print lexical_diversity(tweets_words['screen_names'])
print lexical_diversity(tweets_words['hashtags'])
print average_words(df.texts.str.split(',,,'))

In [40]:
most_tweeted = df[df.retweet_counts!=0][['texts','screen_names','retweet_counts']]
# most_tweeted['texts']= most_tweeted['texts'].apply(lambda x: ' '.join(x))
# most_tweeted['screen_names']= most_tweeted['screen_names'].apply(lambda x: ' '.join(x))
most_tweeted.drop_duplicates(inplace=True)
most_tweeted.sort_values('retweet_counts',ascending=False,inplace=True)
print most_tweeted

In [24]:
_retweets = twitter_api.statuses.retweets(id=317127304981667841)
print [r['user']['screen_name'] for r in _retweets]

In [42]:
df1 = tweets_words['texts'].value_counts()
# df1.columns.name = 'Type'
df1.index.name = 'Word Rank'
df1.plot(loglog=True, legend=False).set(xlabel="x label", ylabel="Freq")
df1.plot(loglog=True, legend=False)

In [43]:
df1 = tweets_words['texts'].value_counts()
for k,v in tweets_words.items():
    v.value_counts().plot(kind='hist').set(ylabel="Number of items in bin", xlabel="Bins (number of times an item appeared)",title= k)
    plt.show()

In [27]:
retweet_counts = df[df.retweet_counts!=0]['retweet_counts']
retweet_counts.plot(kind='hist').set(ylabel="Number of items in bin", xlabel="Bins (number of times an item appeared)",title='retweet_counts')

In [28]:
np.log(retweet_counts).plot(kind='hist').set(ylabel="Number of items in bin", xlabel="Bins (number of times an item appeared)",title='log_retweet_counts')

In [38]:
from datetime import datetime as dt


def my_decorator(orgi_fn):
    fn_name = 'Executing %s' % orgi_fn.__name__

    def new_fn(*args, **kwargs):
        tic = dt.now()  # pre_process
        # original_function (encapsulated with try catch)
        try:
            org_result = orgi_fn(*args, **kwargs)
            org_result = 'and the ans is %s' % float(org_result)  # post process
        except Exception as exp:
            org_result = 'but exception found, "%s"' % exp
        exe_time = dt.now() - tic
        print '%s %s' % (exe_time, fn_name),
        return org_result

    return new_fn


@my_decorator
def add(x, y):
    _ = [_ for _ in range(1000000)]  # delay use time.sleep()
    return x + y


@my_decorator
def mul(x, y):
    _ = [_ for _ in range(1000000)]  # delay
    return x * y


print add(5, 6)  # always return in float format
print mul(5, 6)  # always return in float format
print add('5', 6)  # always return in float format
print add('5', '6')  # always return in float format

In [ ]:
%%timeit
df = statuses_df
df['text']= df.text.apply(lambda x: pproc(x.split()))
entities = dfm.from_records(df.entities)
hashtags = entities.hashtags.apply(lambda x: pproc([x['text'] for x in x]))
screen_name = entities.user_mentions.apply(lambda x: pproc([x['screen_name'] for x in x]))
retweet_screen_name = df.retweeted_status.dropna().apply(lambda x: pproc(x['user']['screen_name'].split()))
p = ['retweet_count', 'text']
df = pd.concat([df[['text','retweet_count']], hashtags, screen_name,retweet_screen_name],axis=1)
df.columns = ['texts','retweet_counts','hashtags','screen_names','retweet_screen_names'] 
df = df[['texts', 'screen_names', 'hashtags', 'retweet_counts', 'retweet_screen_names']].drop_duplicates()

##############################################################################################################################################

%%timeit
df = statuses_df
def z(col):
    nRow = pd.Series()
    if col.name == 'text':
        return col.apply(lambda x: pproc(x.split()))
    elif col.name == 'entities':
        entities = dfm.from_records(col)
        nRow['hashtags'] = entities.hashtags.apply(lambda x: pproc([x['text'] for x in x]))
        nRow['screen_name'] = entities.user_mentions.apply(lambda x: pproc([x['screen_name'] for x in x]))
        return nRow
    elif col.name == 'retweeted_status':
        return col.dropna().apply(lambda x: pproc(x['user']['screen_name'].split()))
    # print nRow.texts
    
df = df[['text','retweeted_status']].apply(z) #  ,'entities','retweeted_status'
# print df

############################################################################################################################################


# %%timeit
# Dead slow approch
df = statuses_df
def z(row):
    nRow = pd.Series()
    nRow['texts'] = pproc(row['text'].split(' '))
    nRow['screen_names'] = pproc([val['screen_name'] for val in row['entities']['user_mentions']])
    nRow['hashtags'] = pproc([val['text'] for val in row['entities']['hashtags']])
    nRow['retweet_counts'] = row['retweet_count']
    nRow['retweet_screen_names'] = np.nan if pd.isnull(row['retweeted_status']) else pproc(row['retweeted_status']['user']['screen_name'].split())
    return nRow
df = df.apply(z,axis=1)



############################################################################################################################################

# %%timeit
# classic way for decoding tweet
pproc = lambda x: ',,,'.join(x).lower()
real = []
for status in statuses:
    temp = {}
    temp['texts'] = pproc(status['text'].split())
    temp['screen_names'] = []
    temp['hashtags'] = []
    temp['retweet_counts'] = status['retweet_count']
    retweeted_status = status.get('retweeted_status')
    if retweeted_status:
        temp['retweet_screen_name'] = pproc([retweeted_status['user']['screen_name']])
    
    for user_mention in status['entities']['user_mentions']:
        temp['screen_names'].append(user_mention['screen_name'] )
    for hashtag in status['entities']['hashtags']:
        temp['hashtags'].append( hashtag['text'] )
    temp['hashtags'] = pproc(temp['hashtags'])
    temp['screen_names'] = pproc(temp['screen_names'])
    
    real.append(temp)
a = dfm(real,columns=['texts','screen_names','hashtags','retweet_counts','retweet_screen_name']).drop_duplicates()
# print a